AIT614 Lab2 by Khanh Nguyen

In [2]:
!pip install pymongo

#Connect Mongodb

In [9]:
import pymongo

myclient = pymongo.MongoClient("mongodb://localhost:27017/")

#mydb = myclient["Lab2_KhanhNguyen"]

In [4]:
print(myclient.list_database_names())

['AIT614', 'admin', 'config', 'local']


#Import libraries

In [5]:
import pymongo
import pandas as pd
import json

#Load the CSV file

In [6]:
df = pd.read_csv("C:/Users/khanh/Desktop/GMUeducation/AIT614/class2/EmployeeAttrition.csv")
df.head(5)

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2


In [7]:
# convert to Json
data = json.loads(df.to_json(orient='records'))

In [8]:
mongo_db = myclient["myDB"]
mongo_db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'myDB')

In [11]:
collection_name = "Employees"

In [12]:
collection = mongo_db[collection_name]

In [13]:
#Insert the data into MongoDB collection
collection.insert_many(data)

#QUERY MONGDB

In [14]:
collection.count_documents({})

1470

1.(5 points) Count the employees whose TotalWorkingYears are greater than 20.
Hint: use count_documents(). The result is 207.

In [16]:
collection.count_documents(
    {"TotalWorkingYears": {'$gt': 20}}
)

207

2. (15 points) Find EmployeeNumber, EducationField, JobRole for all the employees whose Age is between 25 and 30 and Education is 5. Display only EmployeeNumber, EducationField, and JobRobe in the output.

In [97]:
employees = collection.find(
    {"$and": [    
    {"Age": {"$lte": 30, "$gte": 25}},
    {"Education": {"$eq": 5}},
    ]}
)

# print formatted outputs
print('Employee Number', '\t', 'Job Role', '\t\t\t', 'Education Field')
print('----------------------------------------------------------------------')
for em in employees:
    print('{0}\t\t\t {1} \t\t{2}'.format(em['EmployeeNumber'], em['JobRole'], em['EducationField']))

Employee Number 	 Job Role 			 Education Field
----------------------------------------------------------------------
455			 Laboratory Technician 		Other
565			 Research Scientist 		Technical Degree
747			 Sales Executive 		Marketing
1094			 Laboratory Technician 		Life Sciences


3 (15 points) For all the women employees having Age between 35 and 40 and
TotalWorkingYears < 5, sort EmployeeNumber in an ascending order. Print only
Department and EmployeeNumber in the output.


In [96]:
employees = collection.find(
    {"$and": [    
    {'Gender': 'Female',
     "Age": {"$lte": 40, "$gte": 35}},
    {"TotalWorkingYears": {"$lt": 5}}
    ]}
).sort('EmployeeNumber', 1)

# print formatted outputs
print( 'Employee Number', '\t\t', 'Department' )
print('----------------------------------------------------------------------')
for em in employees:
    print('{0}\t\t\t{1}'.format(em['EmployeeNumber'], em['Department'] ))

Employee Number 		 Department
----------------------------------------------------------------------
49			Sales
75			Research & Development
245			Research & Development
805			Sales
1569			Research & Development
1662			Research & Development
1675			Research & Development
1886			Research & Development
2052			Research & Development


4. (20 points) Find employees whose HourlyRate is greater than or equal to 100 or DailyRate
is greater than 1490. Display Age, HourlyRate, DailyRate, and Department only and sort
DailyRate in an ascending order.


In [99]:
employees2 = collection.find(
    {"$or": [    
    {"HourlyRate": {"$gte": 100}},
    {"DailyRate": {"$gt": 1490}}
    ]}
).sort('DailyRate', 1)

# print formatted outputs
print('Age', '\t', 'Hourly Rate', '\t', 'Daily Rate', '\t', 'Department')
print('----------------------------------------------------------------------')
for em in employees2:
    print('{0}\t {1}\t\t {2}\t\t {3}'.format(em['Age'], em['HourlyRate'], em['DailyRate'], em['Department']))

Age 	 Hourly Rate 	 Daily Rate 	 Department
----------------------------------------------------------------------
31	 100		 218		 Sales
29	 100		 224		 Research & Development
45	 100		 306		 Sales
38	 100		 345		 Sales
35	 100		 528		 Human Resources
22	 100		 594		 Research & Development
19	 100		 602		 Sales
26	 100		 652		 Research & Development
34	 100		 702		 Research & Development
32	 100		 976		 Sales
21	 100		 996		 Research & Development
37	 100		 1040		 Research & Development
50	 100		 1046		 Research & Development
36	 100		 1120		 Sales
33	 100		 1198		 Research & Development
32	 100		 1311		 Research & Development
38	 100		 1336		 Human Resources
31	 100		 1445		 Research & Development
40	 100		 1479		 Sales
40	 61		 1492		 Research & Development
38	 76		 1495		 Research & Development
49	 96		 1495		 Research & Development
38	 87		 1495		 Research & Development
29	 41		 1496		 Research & Development
28	 92		 1496		 Sales
39	 44		 1498		 Sales
60	 80		 1499		 Sales


5. (20 points) For each JobRole, find the average MonthlyIncome. Print out the formatted
monthly incomes in hundredth and arrange them in descending order?
Hint: Use $group and $avg. Use {:.2f} and/or other format monthly income.

In [254]:

result = collection.aggregate([
    {"$group" : {
        "_id" : "$JobRole",  
        "Avg_monthly_income" : {"$avg" : '$MonthlyIncome'}
    }},
    {"$sort" : {
        "Avg_monthly_income": -1}
    }
])

# print formatted outputs
print('Job Role', '\t\t\t\t', 'Average Monthly Income')
print('----------------------------------------------------------')
for x in result:
    print(x['_id'],  '\n\t\t\t\t\t', '{:.2f}'.format(x['Avg_monthly_income']))


Job Role 				 Average Monthly Income
----------------------------------------------------------
Manager 
					 17181.68
Research Director 
					 16033.55
Healthcare Representative 
					 7528.76
Manufacturing Director 
					 7295.14
Sales Executive 
					 6924.28
Human Resources 
					 4235.75
Research Scientist 
					 3239.97
Laboratory Technician 
					 3237.17
Sales Representative 
					 2626.00


6. (25 points) Count the different MaritalStatus when Attrition is YES and AGE is greater than
35 in the dataset. Arrange the count in descending order.
Hint: Use $match, $group, $sum, and $sort. 

In [255]:
result1 = collection.aggregate([
    {'$match': {
       "$and": [
        {"Attrition": "Yes"},
        {"Age": {"$gt": 35}}     
       ]
    }}, 
    {"$group" : {
        "_id" : "$MaritalStatus",  
        "count" : {"$sum" : '$EmployeeCount'}
    }},
    {"$sort" : {
        "count": -1}
    }
])
list(result1)

[{'_id': 'Married', 'count': 33},
 {'_id': 'Single', 'count': 30},
 {'_id': 'Divorced', 'count': 14}]